# UAT for CYCLE01: Core Engine & Workflow Foundation

This notebook demonstrates and verifies the User Acceptance Test scenarios for CYCLE01.

In [ ]:
import subprocess
from pathlib import Path

from ase import Atoms

from mlip_autopipec.database import AseDBWrapper

# Clean up previous runs
if Path('uat.db').exists():
    Path('uat.db').unlink()

## UAT-C01-01 & UAT-C01-03: Labeling and Database Verification

**GIVEN:** A clean environment and an unlabeled structure in the database.

In [ ]:
db_wrapper = AseDBWrapper(db_path='uat.db')
atoms = Atoms("H2O", positions=[[0, 0, 0], [1, 0, 0], [0, 1, 0]])
structure_id = db_wrapper.add_atoms(atoms)
print(f"Added structure with ID: {structure_id}")

# Verify initial state is 'unlabeled'
with db_wrapper._connect() as db:
    row = db.get(id=structure_id)
    assert row.key_value_pairs['state'] == 'unlabeled'
    print("Initial state is correctly 'unlabeled'.")

**WHEN:** The user executes the `label` command.

In [ ]:
# We need a mock pw.x command for this to run in a CI environment
mock_qe_script = "#!/bin/bash\ncp tests/integration/dummy_qe_output.in $2"
mock_qe_path = Path('mock_pw.sh')
with open(mock_qe_path, 'w') as f:
    f.write(mock_qe_script)
mock_qe_path.chmod(0o755)

# Create a dummy output file for the mock script to copy
dummy_output_content = DUMMY_QE_OUTPUT = """\n     Final energy            =     -17.83... Ry\n!    total energy              =     -17.83... Ry\n\nForces acting on atoms (cartesian axes, Ry/au):\n\n     atom    1   type  H   force =     0.000...     0.000...     0.000...\n     atom    2   type  O   force =     0.000...     0.000...     0.000...\n\nTotal stress (Ry/bohr**3):\n     -0.000...     0.000...     0.000...\n      0.000...    -0.000...     0.000...\n      0.000...     0.000...    -0.000...\n"""
with open('tests/integration/dummy_qe_output.in', 'w') as f:
    f.write(dummy_output_content)

result = subprocess.run(f'cdd label --id {structure_id}', shell=True, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

**THEN:** The database state is updated to 'labeled' and contains the correct results.

In [ ]:
with db_wrapper._connect() as db:
    row = db.get(id=structure_id)
    assert row.key_value_pairs['state'] == 'labeled'
    print("Final state is correctly 'labeled'.")
    
    dft_result_json = row.key_value_pairs['dft_result']
    print(f"Retrieved DFT result (JSON): {dft_result_json}")
    assert 'energy' in dft_result_json

## UAT-C01-02: Training Step Execution

**WHEN:** The user executes the `train` command.

In [ ]:
result = subprocess.run('cdd train', shell=True, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

**THEN:** The system reports that training is not yet implemented.

In [ ]:
assert "Training not implemented" in result.stdout